### Разработка нейросетевых функций. Операция Convolution 3D

#### Описание параметров

in_channels (int) – Number of channels in the input image\
out_channels (int) – Number of channels produced by the convolution\
kernel_size (int or tuple) – Size of the convolving kernel\
stride (int or tuple, optional) – Stride of the convolution. Default: 1\
padding (int, tuple or str, optional) – Padding added to all four sides of the input. Default: 0\
padding_mode (str, optional) – 'zeros', 'reflect', 'replicate' or 'circular'. Default: 'zeros'\
dilation (int or tuple, optional) – Spacing between kernel elements. Default: 1\
groups (int, optional) – Number of blocked connections from input channels to output channels. Default: 1\
bias (bool, optional) – If True, adds a learnable bias to the output. Default: True\

In [1]:
import torch
import numpy as np
def myConv3d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True,
             padding_mode='zeros'):
    def Conv(matrix):
        if (in_channels%groups != 0) or (out_channels%groups!=0):
            raise Exception('in_channel и out_channel не делятся на группы!')

        if bias == True:
            bias_val = torch.rand(out_channels)
        else:
            bias_val = torch.zeros(out_channels)

        if (padding_mode == 'zeros'):
            pad = torch.nn.ZeroPad3d(padding)
            matrix = pad(matrix)
        if (padding_mode == 'reflect'):
            pad = torch.nn.ReflectionPad3d(padding)
            matrix = pad(matrix)
        if (padding_mode == 'replicate'):
            pad = torch.nn.ReplicationPad3d(padding)
            matrix = pad(matrix)
        if (padding_mode == 'circular'):
            pad = torch.nn.CircularPad3d(padding)
            matrix = pad(matrix)

        if type(kernel_size) == tuple:
            filter = torch.rand(out_channels, in_channels//groups, kernel_size[0], kernel_size[1], kernel_size[2])
        if type(kernel_size) == int:
            filter = torch.rand(out_channels, in_channels//groups, kernel_size, kernel_size, kernel_size)

        li = []
        for l in range(out_channels):
            feature_map = np.array([]) 
            for k in range (0, matrix.shape[1]-((filter.shape[2]-1)*dilation+1)+1, stride):
                for i in range (0, matrix.shape[2]-((filter.shape[3]-1)*dilation+1)+1, stride):
                    for j in range (0, matrix.shape[3]-((filter.shape[4]-1)*dilation+1)+1, stride):
                        summa = 0
                        for c in range (in_channels//groups): 
                            if groups>1:
                                val = matrix[l*(in_channels//groups)+c][k:k+(filter.shape[2]-1)*dilation+1:dilation, 
                                                                        i:i+(filter.shape[3]-1)*dilation+1:dilation,
                                                                        j:j+(filter.shape[4]-1)*dilation+1:dilation]
                            else:
                                val = matrix[c][k:k+(filter.shape[2]-1)*dilation+1:dilation, i:i+(filter.shape[3]-1)*
                                                dilation+1:dilation, j:j+(filter.shape[4]-1)*dilation+1:dilation]
                            mini_sum = (val*filter[l][c]).sum()
                            summa = summa + mini_sum
                        feature_map = np.append(feature_map, float(summa + bias_val[l])) #bias

            li.append(feature_map.reshape((matrix.shape[1]-((filter.shape[2]-1)*dilation+1))//stride+1,
                                          (matrix.shape[2]-((filter.shape[3]-1)*dilation+1))//stride+1, 
                                          (matrix.shape[3]-((filter.shape[4]-1)*dilation+1))//stride+1))

        return np.array(li), torch.tensor(np.array(filter)), torch.tensor(np.array(bias_val))

    return Conv

### Тестирование функции

In [2]:
img1 = torch.rand(4, 6, 6, 6)
img2 = torch.rand(3, 24, 28, 20)
img3 = torch.rand(1, 1, 1, 1)

In [3]:
Func = myConv3d(in_channels=4, out_channels=2, kernel_size=3, stride=1, padding=0, dilation=1, groups=2,
                bias=True, padding_mode='zeros')
result, kernel, bias_val = Func(img1)
torchFunction = torch.nn.Conv3d(in_channels=4, out_channels=2, kernel_size=3, stride=1, padding=0, dilation=1, groups=2,
                                bias=True, padding_mode='zeros')
torchFunction.weight.data = kernel
torchFunction.bias.data = bias_val
myResult = str(np.round(result, 2))
torchResult = str(np.round(np.array(torchFunction(img1).data), 2))
torchResult == myResult

True

In [4]:
Func2 = myConv3d(in_channels=3, out_channels=1, kernel_size=3, stride=1, padding=0, dilation=2, groups=1, bias=True,
                 padding_mode='zeros')
result2, kernel2, bias_val2 = Func2(img2)
torchFunction2 = torch.nn.Conv3d(in_channels=3, out_channels=1, kernel_size=3, stride=1, padding=0, dilation=2, 
                                 groups=1, bias=True, padding_mode='zeros')
torchFunction2.weight.data = kernel2
torchFunction2.bias.data = bias_val2
myResult = str(np.round(result2, 2))
torchResult = str(np.round(np.array(torchFunction2(img2).data), 2))
torchResult == myResult

True

In [5]:
Func3 = myConv3d(in_channels=1, out_channels=1, kernel_size=1, stride=1, padding=0, dilation=1, groups=1, bias=True,
                 padding_mode='zeros')
result3, kernel3, bias_val3 = Func3(img3)
torchFunction3 = torch.nn.Conv3d(in_channels=1, out_channels=1, kernel_size=1, stride=1, padding=0, dilation=1,
                                 groups=1, bias=True, padding_mode='zeros')
torchFunction3.weight.data = kernel3
torchFunction3.bias.data = bias_val3
myResult = str(np.round(result3, 2))
torchResult = str(np.round(np.array(torchFunction3(img3).data), 2))
torchResult == myResult

True